# Qarameterized circuits: Quantum parameters for QML
**Evan Peters**, University of Waterloo, Institute for Quantum Computing, Fermilab Quantum Institute 

**Prasanth Shyamsundar**, Fermi National Accelerator Laboratory, Fermilab Quantum Institute

GitHub repository: [https://github.com/peterse/groveropt](https://github.com/peterse/groveropt)

---

<span style="color:gray"> *So you want to train your quantum circuit but you're tired of optimizing everything **classically**...?*</span>.

Variational quantum circuits are the foundation of many near term quantum algorithms, from quantum chemistry to quantum machine learning. The applications span from drug discovery 💊 to particle physics ☢ to quantitative finance 🤑 and beyond!. In most of these applications, parameters are optimized by using the quantum quantum circuit to compute a cost function for some initial choice of parameters, and then updating these parameters using a **classical** optimizer running in lockstep with the quantum circuits.

In this project we introduce **Quantum-parameterized (Qarameterized) circuits** for the optimization of variational circuits, which turns the power of a quantum circuit back towards itself to boost optimization!


---

### Background

Training a parameterized quantum circuit requires that we find some set of classical angles $\theta$ that correspond to the extremum of an observable. For some parameterized circuit $U(\theta)$ with parameters $\theta$, our goal is usually to find parameters that minimize the value of some observable quantity $H$, that is

$$
\underset{\theta}{\operatorname{argmin}} \, \langle 0 | U(\theta) H U^\dagger (\theta)|0\rangle
$$

However, if we construct a modified observable $H_Q$ that's accessible _in quantum space_ and we upload our parameter values into quantum states $|\theta\rangle$, then we can rephrase this as a quantum parameterization problem:

$$
\underset{|\theta\rangle}{\operatorname{argmin}} \, \langle \theta | H_Q |\theta\rangle
$$

Our technique to achieving quantum parameterization is summarized in the following animation:

<center> <img src="img/lifting_animation/lifting_animation.png" width=1500 /> </center>

<h3><span style="color:gray"> ...Now lets find out how this works!</span></h3> 

---

### 1. What is a Qarameterized circuit?

A Qarameterized circuit is a variational circuit that is parameterized by control quantum registers (possibly in addition to classical free parameters). Quantum parameterization has been proposed previously, for example [(Verdon, 2018)](https://arxiv.org/abs/1806.09729) and [(Shyamsundar, 2021)](https://arxiv.org/abs/2102.04975), but our algorithm improves the probability of finding circuit optima using a *gradient-free technique*.

In order to apply this optimization technique we prepare the circuit by "lifting" each parameter into quantum state space:

* The originally classically-parameterized rotation gates are replaced by controlled rotation gates, with the quantum parameters serving as controls.
* The computational basis states of the control register correspond to discrete values of $\theta = \sum_k \theta_k 2^{-k}$ in the range $[0, 2\pi)$.
* Setting the state of the quantum parameters to one of the computational basis states recovers the behavior of the corresponding classically-parameterized circuit.

<center><img src="img/lifting_diagram/lifting_diagram.png" width=500/></center>

---

<h3>2. Why use a Qarameterized Circuit?</h3>
    
<center><h1> <span style="color:#a88570;">.</span><span style="color:#a48976;">.</span><span style="color:#a08e7d;">.</span><span style="color:#9c9283;">S</span><span style="color:#98968a;">u</span><span style="color:#949b90;">p</span><span style="color:#909f96;">e</span><span style="color:#8ca39d;">r</span><span style="color:#89a8a3;">p</span><span style="color:#85aca9;">o</span><span style="color:#81b0b0;">s</span><span style="color:#7db4b6;">i</span><span style="color:#79b9bd;">t</span><span style="color:#75bdc3;">i</span><span style="color:#71c1c9;">o</span><span style="color:#6dc6d0;">n</span><span style="color:#69cad6;">!</span> </h1></center>

The main advantage of using quantum parameters is that they can exist in a superposition of the computational basis states. This opens up the possibility of finding good/optimal states for the quantum parameters in a manifestly quantum manner.

#### Oracles to evaluate the states of the quantum parameters

In order to optimize the values of parameters, classical or quantum, we need a way to evaluate the parameter values (or states). In order to leverage the superposition aspect of Qarameterized circuits discussed above, we need to _coherently_ evaluate the states of the quantum parameters. Similar to Grover's algorithm, we need to construct an oracle to apply a state-dependent phase to the quantum parameters.


In this project, we set the goal of the optimization is to find train the variational circuit to produce the all-zeros bitstring (similar to the [Pennylane Barren Plateus Tutorial](https://pennylane.ai/qml/demos/tutorial_barren_plateaus.html](https://pennylane.ai/qml/demos/tutorial_barren_plateaus.html) ). We created an oracle which evaluates the quantum parameters based on the following score function:
$$\mathrm{score} = \Big|\big\langle\psi(\theta_\mathrm{quant})\big|0\big\rangle\Big|^2$$

<span style="color:gray"> For more information about this implementation, check out the notebook `more_detailed.ipynb` in our [GitHub repository](https://github.com/peterse/groveropt) </span>

---

### 3. Training Qarameterized circuits using amplitude amplification

[(Shyamsundar, 2021)](https://arxiv.org/abs/2102.04975) introduces the non-boolean amplitude amplification algorithm, which can preferentially amplify the states of the quantum parameters based on an objective function. Here we simply apply the algorithm to amplify the "good states" in the quantum control registers. Our algorithm queries the cost landscape for an observable $H$ *for a  superposition of choices of $|\theta\rangle$*. Specifically, this happens in two steps:

1. Construct a quantum oracle to coherently evaluate the state of the control registers, using $\exp (i H)$.
    
2. Train the circuit using a modified version of Grover's algorithm, which preferentially amplifies the good states of $|\theta\rangle$ held in the control registers.



---

### 4. Measuring the quantum parameter

Finally, measuring the state of the control register after the amplification process will yield good states for the quantum parameters. This completes the manifestly-quantum, gradient-free training of our Qarameterized circuits!

<center> <h6 style="font-size:30px">&#x1F389; &#x1F389; &#x1F389; </h6> </center>
 	

---

## Experiments

As a warm-up, we begin by creating a low-depth random variational circuit with two trainable parameters. The classical quantum circuit we used for this experiment is shown below, alongside its qarameterized counterpart that has been lifted to use quantum parameters.




picture..................

We then used qarameterization to search the parameter space for the optimal choices of $(\theta_1, \theta_2)$. Since this circuit has only two trainable parameters we can easilyvisualize the score landscape for different values of the parameters, compared to the probability that qarameterization samples a given choice of $(\theta_1, \theta_2)$

<center> <img src="img/plot1.png" width=800/></center>

<center><h3> &#128293;&#128293;<span style="color:#fff800;">.</span><span style="color:#fff500;">.</span><span style="color:#fff200;">.</span><span style="color:#ffef00;">T</span><span style="color:#ffec00;">h</span><span style="color:#ffea00;">e</span><span style="color:#ffe700;"> </span><span style="color:#ffe400;">p</span><span style="color:#ffe100;">a</span><span style="color:#ffde00;">r</span><span style="color:#ffdb00;">a</span><span style="color:#ffd800;">m</span><span style="color:#ffd500;">e</span><span style="color:#ffd300;">t</span><span style="color:#ffd000;">e</span><span style="color:#ffcd00;">r</span><span style="color:#ffca00;"> </span><span style="color:#ffc700;">p</span><span style="color:#ffc400;">r</span><span style="color:#ffc100;">o</span><span style="color:#ffbe00;">b</span><span style="color:#ffbb00;">a</span><span style="color:#ffb900;">b</span><span style="color:#ffb600;">i</span><span style="color:#ffb300;">l</span><span style="color:#ffb000;">i</span><span style="color:#ffad00;">t</span><span style="color:#ffaa00;">y</span><span style="color:#ffa700;"> </span><span style="color:#ffa400;">l</span><span style="color:#ffa100;">a</span><span style="color:#ff9f00;">n</span><span style="color:#ff9c00;">d</span><span style="color:#ff9900;">s</span><span style="color:#ff9600;">c</span><span style="color:#ff9300;">a</span><span style="color:#ff9000;">p</span><span style="color:#ff8d00;">e</span><span style="color:#ff8a00;"> </span><span style="color:#ff8800;">a</span><span style="color:#ff8500;">l</span><span style="color:#ff8200;">m</span><span style="color:#ff7f00;">o</span><span style="color:#ff7c00;">s</span><span style="color:#ff7900;">t</span><span style="color:#ff7600;"> </span><span style="color:#ff7300;">p</span><span style="color:#ff7000;">e</span><span style="color:#ff6d00;">r</span><span style="color:#ff6a00;">f</span><span style="color:#ff6700;">e</span><span style="color:#ff6400;">c</span><span style="color:#ff6100;">t</span><span style="color:#ff5e00;">l</span><span style="color:#ff5c00;">y</span><span style="color:#ff5900;"> </span><span style="color:#ff5600;">r</span><span style="color:#ff5300;">e</span><span style="color:#ff5000;">p</span><span style="color:#ff4d00;">r</span><span style="color:#ff4a00;">o</span><span style="color:#ff4700;">d</span><span style="color:#ff4400;">u</span><span style="color:#ff4100;">c</span><span style="color:#ff3e00;">e</span><span style="color:#ff3b00;">s</span><span style="color:#ff3800;"> </span><span style="color:#ff3500;">t</span><span style="color:#ff3200;">h</span><span style="color:#ff2f00;">e</span><span style="color:#ff2c00;"> </span><span style="color:#ff2900;">c</span><span style="color:#ff2600;">o</span><span style="color:#ff2300;">s</span><span style="color:#ff2000;">t</span><span style="color:#ff1e00;"> </span><span style="color:#ff1b00;">l</span><span style="color:#ff1800;">a</span><span style="color:#ff1500;">n</span><span style="color:#ff1200;">d</span><span style="color:#ff0f00;">s</span><span style="color:#ff0c00;">c</span><span style="color:#ff0900;">a</span><span style="color:#ff0600;">p</span><span style="color:#ff0300;">e</span><span style="color:#ff0000;">!</span> &#128293;&#128293; </h3> </center>


### Scaling it up

To put our algorithm to the test, we constructed and lifted a deeper 4-qubit circuit with four trainable parameters:

<center> <img src="img/experiment2.png" width=1300 /> </center>

These kinds of circuits are quite powerful regardless of the single parameter per layer, for example being able to serve as universal function approximators [(Schuld, 2020)](https://arxiv.org/abs/2008.08605)! We used the Qarameterized circuit as an initializer for training an arbitrary cost function, and found that this technique greatly improved the initial cost of the circuit, allowing gradient-based optimizers to finish the job quickly and smoothly:

<center><img src="img/plot2_anno.png" width=600/> </center>

---

## Discussion

* We can see from the plots above that the amplification process has increased the probabilities of the good values of $(\theta_0, \theta_1)$, i.e., parameter-values with higher scores.
* 
